In [35]:
function supnorm(u) 
    s = 0 
    for entry in u 
        s = max(abs(entry),s) 
    end 
    return s 
end 

function inner_product(u::Array{<:Real,1}, v::Array{<:Real,1}) 
    s = 0 
    n = size(u)[1] 
    for i in 1:n
        s += u[i] * v[i] 
    end 
    return s
end

LoadError: UndefVarError: IntegerArray not defined

In [ ]:
function power_method(A::Array{<:Real,2}, x0::Array{<:Real,1})
    y = A*x0
    x = y / supnorm(y)
    return y
end

function rayleigh(A::Array{<:Real,2}, x0::Array{<:Real,1})
    lambda = inner_product(x0, A*x0) / inner_product(x0, x0)
    return lambda
end

In [103]:
function power_method_iterate(A::Array{<:Real,2}, x::Array{<:Real,1}, iterations::Int)
    for i in 1:iterations
        x = power_method(A, x)
    end
    lambda = rayleigh(A, x)
    return x, lambda
end 

power_method_iterate (generic function with 3 methods)

In [104]:
A = [0 0 1 1/2; 1/3 0 0 0; 1/3 1/2 0 1/2; 1/3 1/2 0 0]

4×4 Matrix{Float64}:
 0.0       0.0  1.0  0.5
 0.333333  0.0  0.0  0.0
 0.333333  0.5  0.0  0.5
 0.333333  0.5  0.0  0.0

In [105]:
x0 = ones(4) / 4
x0 = [1; 0; 0; 0]

4-element Vector{Int64}:
 1
 0
 0
 0

In [106]:
power_method_iterate(A, x0, 100)[1]

4-element Vector{Float64}:
 0.3870967741935477
 0.1290322580645159
 0.2903225806451608
 0.19354838709677386

In [107]:
power_method_iterate(A, x0, 100)[2]

1.0

In [38]:
sortperm(power_method_iterate(A, x0, 100)[1], rev=true)

4-element Vector{Int64}:
 1
 3
 4
 2

In [143]:
function check_unit_columns(A::Array{<:Real,2})
    n = size(A)[1]
    
    for j in 1:n
        s = 0
        for i in 1:n
            s += A[i, j]
        end
        println("Column ", j, ": ", s)
    end
end

function fill_columns(A::Array{<:Real,2})
    n = size(A)[1]
    
    B = zeros(size(A)[1], size(A)[2])
    
    for j in 1:n
        s = 0
        for i in 1:n
            s += A[i, j]
            B[i, j] += A[i, j]
        end
        if s < 1
            to_add = (1 - s) / n
            for i in 1:n
                B[i, j] += to_add
            end
        end
    end
    
    return B
end

B = [1. 0 0; 0 0 0; 0 0 0.5]

println(fill_columns(A))
println(fill_columns(B))

check_unit_columns(fill_columns(A))
check_unit_columns(fill_columns(B))

function teleport(A::Array{<:Real,2}, damp::Real=0.15)
    A = fill_columns(A)
    return (1 - damp)*A + damp*(ones(size(A)[1], size(A)[2]) / size(A)[1])
end

println(teleport(A))
println(teleport(B))

check_unit_columns(teleport(A))
check_unit_columns(teleport(B))

println(A)
println(B)

function page_rank(A::Array{<:Real,2}, x::Array{<:Real,1}=[NaN], iterations::Int=100, page_names=[nothing], 
        damp::Real=NaN, fill_cols::Bool=false)
    println(A)
    
    if !isnan(damp)
        A = teleport(A, damp)
    elseif fill_cols
        A = fill_columns(A)
    end
    
    println(A)
    
    if isnan(x[1])
        x = ones(size(A)[1]) / size(A)[1]
        println(x)
    end
    
    for i in 1:iterations
        x = power_method(A, x)
    end
    lambda = rayleigh(A, x)
    
    println("Largest eigenvalue (should be 1): ", lambda)
    println("Page rank weight eigenvector: ", x)
    if isnothing(page_names[1])
        println("Pages in order from most to least important: ", sortperm(x, rev=true)) 
    else 
        println("Pages in order from most to least important: ", page_names[sortperm(x, rev=true)]) 
    end
    
    return x, lambda
end 

[0.0 0.0 1.0 0.5; 0.3333333333333333 0.0 0.0 0.0; 0.3333333333333333 0.5 0.0 0.5; 0.3333333333333333 0.5 0.0 0.0]
[1.0 0.3333333333333333 0.16666666666666666; 0.0 0.3333333333333333 0.16666666666666666; 0.0 0.3333333333333333 0.6666666666666666]
Column 1: 1.0
Column 2: 1.0
Column 3: 1.0
Column 4: 1.0
Column 1: 1.0
Column 2: 1.0
Column 3: 1.0
[0.0375 0.0375 0.8875 0.46249999999999997; 0.3208333333333333 0.0375 0.0375 0.0375; 0.3208333333333333 0.46249999999999997 0.0375 0.46249999999999997; 0.3208333333333333 0.46249999999999997 0.0375 0.0375]
[0.9 0.3333333333333333 0.19166666666666665; 0.049999999999999996 0.3333333333333333 0.19166666666666665; 0.049999999999999996 0.3333333333333333 0.6166666666666667]
Column 1: 0.9999999999999999
Column 2: 1.0
Column 3: 0.9999999999999999
Column 4: 0.9999999999999999
Column 1: 1.0
Column 2: 1.0
Column 3: 1.0
[0.0 0.0 1.0 0.5; 0.3333333333333333 0.0 0.0 0.0; 0.3333333333333333 0.5 0.0 0.5; 0.3333333333333333 0.5 0.0 0.0]
[1.0 0.0 0.0; 0.0 0.0 0.0; 0

page_rank (generic function with 6 methods)

In [144]:
page_rank(A, x0, 100, ["Google" "Apple" "Sony" "Jamba Juice"])[1]

[0.0 0.0 1.0 0.5; 0.3333333333333333 0.0 0.0 0.0; 0.3333333333333333 0.5 0.0 0.5; 0.3333333333333333 0.5 0.0 0.0]
[0.0 0.0 1.0 0.5; 0.3333333333333333 0.0 0.0 0.0; 0.3333333333333333 0.5 0.0 0.5; 0.3333333333333333 0.5 0.0 0.0]
Largest eigenvalue (should be 1): 1.0
Page rank weight eigenvector: [0.3870967741935477, 0.1290322580645159, 0.2903225806451608, 0.19354838709677386]
Pages in order from most to least important: ["Google", "Sony", "Jamba Juice", "Apple"]


4-element Vector{Float64}:
 0.3870967741935477
 0.1290322580645159
 0.2903225806451608
 0.19354838709677386

In [145]:
page_rank(A)[1]

[0.0 0.0 1.0 0.5; 0.3333333333333333 0.0 0.0 0.0; 0.3333333333333333 0.5 0.0 0.5; 0.3333333333333333 0.5 0.0 0.0]
[0.0 0.0 1.0 0.5; 0.3333333333333333 0.0 0.0 0.0; 0.3333333333333333 0.5 0.0 0.5; 0.3333333333333333 0.5 0.0 0.0]
[0.25, 0.25, 0.25, 0.25]
Largest eigenvalue (should be 1): 1.0
Page rank weight eigenvector: [0.38709677419354754, 0.12903225806451585, 0.29032258064516064, 0.19354838709677377]
Pages in order from most to least important: [1, 3, 4, 2]


4-element Vector{Float64}:
 0.38709677419354754
 0.12903225806451585
 0.29032258064516064
 0.19354838709677377

In [146]:
page_rank(B, [NaN], 100, ["Google" "Apple" "Sony" "Jamba Juice"], 0.15)[1]

[1.0 0.0 0.0; 0.0 0.0 0.0; 0.0 0.0 0.5]
[0.9 0.3333333333333333 0.19166666666666665; 0.049999999999999996 0.3333333333333333 0.19166666666666665; 0.049999999999999996 0.3333333333333333 0.6166666666666667]
[0.3333333333333333, 0.3333333333333333, 0.3333333333333333]
Largest eigenvalue (should be 1): 1.0
Page rank weight eigenvector: [0.7087827426810486, 0.10631741140215725, 0.1848998459167952]
Pages in order from most to least important: ["Google", "Sony", "Apple"]


3-element Vector{Float64}:
 0.7087827426810486
 0.10631741140215725
 0.1848998459167952

In [147]:
page_rank(B, [NaN], 100, ["Google" "Apple" "Sony" "Jamba Juice"], NaN)[1]

[1.0 0.0 0.0; 0.0 0.0 0.0; 0.0 0.0 0.5]
[1.0 0.0 0.0; 0.0 0.0 0.0; 0.0 0.0 0.5]
[0.3333333333333333, 0.3333333333333333, 0.3333333333333333]
Largest eigenvalue (should be 1): 1.0
Page rank weight eigenvector: [0.3333333333333333, 0.0, 2.629536350736706e-31]
Pages in order from most to least important: ["Google", "Sony", "Apple"]


3-element Vector{Float64}:
 0.3333333333333333
 0.0
 2.629536350736706e-31

In [149]:
C = [0 0 0; 0 0 0; 1. 1. 0]
page_rank(C, [NaN], 100, ["Google" "Apple" "Sony" "Jamba Juice"], NaN)[1]

[0.0 0.0 0.0; 0.0 0.0 0.0; 1.0 1.0 0.0]
[0.0 0.0 0.0; 0.0 0.0 0.0; 1.0 1.0 0.0]
[0.3333333333333333, 0.3333333333333333, 0.3333333333333333]
Largest eigenvalue (should be 1): NaN
Page rank weight eigenvector: [0.0, 0.0, 0.0]
Pages in order from most to least important: ["Google", "Apple", "Sony"]


3-element Vector{Float64}:
 0.0
 0.0
 0.0

In [150]:
page_rank(C, [NaN], 100, ["Google" "Apple" "Sony" "Jamba Juice"], NaN, true)[1]

[0.0 0.0 0.0; 0.0 0.0 0.0; 1.0 1.0 0.0]
[0.0 0.0 0.3333333333333333; 0.0 0.0 0.3333333333333333; 1.0 1.0 0.3333333333333333]
[0.3333333333333333, 0.3333333333333333, 0.3333333333333333]
Largest eigenvalue (should be 1): 1.0
Page rank weight eigenvector: [0.19999999999999943, 0.19999999999999943, 0.5999999999999983]
Pages in order from most to least important: ["Sony", "Google", "Apple"]


3-element Vector{Float64}:
 0.19999999999999943
 0.19999999999999943
 0.5999999999999983